# Load data

In [ ]:
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')
    path_to_file = '/content/drive/MyDrive/DSA4212_Assignment2/datasets'
    print(path_to_file)
    # move to Google Drive directory
    os.chdir(path_to_file)
    !pwd

Mounted at /content/drive
/content/drive/MyDrive/DSA4212_Assignment2/datasets
/content/drive/MyDrive/DSA4212_Assignment2/datasets


In [ ]:
from sklearn.datasets import dump_svmlight_file
import numpy as np
import pandas as pd
import os
import urllib
import zipfile
from sklearn.model_selection import train_test_split
import shutil
 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

%matplotlib inline

ratings = pd.read_table("ratings.dat",sep = '::',header=None,engine='python',names=['user_id','movie_id',"rating","timestamp"])
users = pd.read_table("users.dat",sep = '::',header=None,engine='python',names =['user_id','gender','age','occupation','zipcode'])
movies = pd.read_table('movies.dat', sep='::', header=None, engine='python', encoding='ISO-8859-1').to_numpy()


#movies.columns = ['movie_id','title','genres']

In [ ]:
ratings.shape

(1000209, 4)

In [ ]:
users.shape

(6040, 5)

In [ ]:
ratings.head(5)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
users.head(5)

,user_id,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
users.info()

In [ ]:
movies

In [ ]:
movies_df=[]
for i in movies:
  movie = []
  movie.append(i[0])
  movie.append(i[1])
  movie.append(i[2])
  movies_df.append(movie)

In [ ]:
movies_df = pd.DataFrame(movies_df, columns=['movie_id','movie_title','genre'])

In [ ]:
movies_df.head(5)

,movie_id,movie_title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movie_only = movies_df[['movie_id','movie_title']]

In [ ]:
movies

array([[1, 'Toy Story (1995)', "Animation|Children's|Comedy"],
       [2, 'Jumanji (1995)', "Adventure|Children's|Fantasy"],
       [3, 'Grumpier Old Men (1995)', 'Comedy|Romance'],
       ...,
       [3950, 'Tigerland (2000)', 'Drama'],
       [3951, 'Two Family House (2000)', 'Drama'],
       [3952, 'Contender, The (2000)', 'Drama|Thriller']], dtype=object)

In [ ]:
data = pd.crosstab(ratings.user_id, ratings.movie_id, values=ratings.rating, aggfunc=sum).fillna(0)

In [ ]:
data

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for col in data.columns:
    data[col] = data[col].astype('category')

In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 8.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630145 sha256=d57ce0a146bbd9c14ca1570f5e850393afca3d41fa14486bb42f84ee870bb6d9
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from sklearn.metrics import mean_absolute_error
from surprise.model_selection import train_test_split

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

data = Dataset.load_builtin(name='ml-1m', prompt=False)

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-1m.zip...
Done! Dataset ml-1m has been saved to /root/.surprise_data/ml-1m


In [ ]:
algo = SVD()

In [ ]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8716  0.8718  0.8754  0.8740  0.8750  0.8736  0.0016  
MAE (testset)     0.6846  0.6846  0.6873  0.6862  0.6863  0.6858  0.0010  
Fit time          63.54   47.08   49.06   48.58   49.16   51.49   6.07    
Test time         2.42    2.28    2.56    2.80    2.74    2.56    0.19    


{'fit_time': (63.54099750518799,
  47.08493900299072,
  49.06336164474487,
  48.584195137023926,
  49.16475248336792),
 'test_mae': array([0.68457402, 0.68463884, 0.68730005, 0.68616128, 0.68629379]),
 'test_rmse': array([0.87162196, 0.87184001, 0.87544516, 0.87402333, 0.87500706]),
 'test_time': (2.419654607772827,
  2.280654191970825,
  2.5613675117492676,
  2.799720287322998,
  2.739182949066162)}

In [ ]:


trainset, testset = train_test_split(data, test_size=.1)

In [ ]:
algo = SVD()

algo.fit(trainset)
#algo.fit(data)
predictions = algo.test(testset)

accuracy.rmse(predictions)

RMSE: 0.8698


0.8697972016878109

In [ ]:
algo.predict('1', '5001')

3.867201420512513

In [ ]:
def get_top10_recommendation(user_id):
  estimated_ratings= []
  for i in range(6041):
    estimated_ratings.append([i+1,algo.predict(str(user_id),str(i)).est])
  ratings_user = ratings[ratings.user_id == user_id]
  movie_list = ratings_user.movie_id.tolist()
  for j in estimated_ratings:
    if j[0] in movie_list:
      estimated_ratings.remove(j)
  estimate = pd.DataFrame(estimated_ratings,columns = ['movie_id','estimated_rating'])
  sorted = estimate.sort_values(['estimated_rating', 'movie_id',], ascending=False).head(10).copy()
  movies=pd.merge(sorted,movie_only,right_on='movie_id',left_index=True)
  return movies[['movie_id','movie_title','estimated_rating']]

In [ ]:
get_top10_recommendation(2)

,movie_id,movie_title,estimated_rating
341,345,"Adventures of Priscilla, Queen of the Desert, ...",4.861827
306,309,"Red Firecracker, Green Firecracker (1994)",4.726592
1882,1951,Oliver! (1968),4.720255
503,507,"Perfect World, A (1993)",4.715307
876,888,Land Before Time III: The Time of the Great Gi...,4.618802
1672,1721,Titanic (1997),4.559073
914,926,All About Eve (1950),4.544464
105,107,Muppet Treasure Island (1996),4.538860
251,254,Jefferson in Paris (1995),4.502223
863,874,Killer: A Journal of Murder (1995),4.494218


In [ ]:
from surprise import KNNBasic
algo2 = KNNBasic()

algo2.fit(trainset)
#algo.fit(data)
predictions2 = algo2.test(testset)

accuracy.rmse(predictions2)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9195


0.9194817166302899

In [ ]:
cross_validate(algo2, data, measures=['RMSE'], cv=5, verbose=False)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'fit_time': (50.76163411140442,
  46.741506814956665,
  52.89540958404541,
  51.96324825286865,
  57.168129444122314),
 'test_rmse': array([0.92222545, 0.92381968, 0.92552888, 0.92105694, 0.92246067]),
 'test_time': (176.43061208724976,
  169.55118680000305,
  183.36666107177734,
  178.56559538841248,
  175.40915036201477)}

In [ ]:
algo2.predict('1', '5001').est

3.5807597968424374

In [ ]:
knn = KNNBasic()
knn.fit(data2)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
def get_top20_recommendation2(user_id):
  estimated_ratings= []
  for i in range(6041):
    estimated_ratings.append([i+1,knn.predict(str(user_id),str(i)).est])
  ratings_user = ratings[ratings.user_id == user_id]
  movie_list = ratings_user.movie_id.tolist()
  for j in estimated_ratings:
    if j[0] in movie_list:
      estimated_ratings.remove(j)
  estimate = pd.DataFrame(estimated_ratings,columns = ['movie_id','estimated_rating'])
  sorted = estimate.sort_values(['estimated_rating', 'movie_id',], ascending=False).head(20).copy()
  movies=pd.merge(sorted,movie_only,right_on='movie_id',left_index=True)
  #movies = pd.merge(estimate, sorted, left_on='movie_id',right_index=True)
  return movies[['movie_id','movie_title','estimated_rating']]

In [ ]:
get_top20_recommendation2(2)

,movie_id,movie_title,estimated_rating
3691,3760,"Kentucky Fried Movie, The (1977)",5.000000
3470,3539,"Filth and the Fury, The (2000)",5.000000
3422,3491,My Chauffeur (1986),5.000000
3202,3271,Of Mice and Men (1992),5.000000
3101,3170,Hi-Yo Silver (1940),5.000000
2995,3064,Poison Ivy: New Seduction (1997),5.000000
2322,2391,"Simple Plan, A (1998)",5.000000
1710,1765,"Letter From Death Row, A (1998)",5.000000
949,961,Little Lord Fauntleroy (1936),5.000000
752,762,Striptease (1996),5.000000
